# Model Service for Code Generation 

In this section, we demonstrate how to deploy a code generation service that provides a REST API endpoint for generating code based on natural language queries. This service provides a REST API endpoint for generating code based on natural language queries, 
The service includes performance optimization features to handle large repositories:
- **Parameter-based operation control** with `metadata_only` mode for fast processing 
- **Resource optimization** with batched processing and configurable timeouts

# Notebook Overview

- Start Execution
- Install and Import Libraries
- Configure Settings
- Register and Log the Model to MLFlow
- API Usage Examples

# Start Execution

In [1]:
import logging
import time

# Configure logger
logger: logging.Logger = logging.getLogger("register_model_logger")
logger.setLevel(logging.INFO)
logger.propagate = False  # Prevent duplicate logs from parent loggers

# Set formatter
formatter: logging.Formatter = logging.Formatter(
    fmt="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)

# Configure and attach stream handler
stream_handler: logging.StreamHandler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

In [2]:
start_time = time.time()  

logger.info("Notebook execution started.")

2025-08-08 11:55:58 - INFO - Notebook execution started.


# Install and Import Libraries

In [3]:
%%time

%pip install -r ../requirements.txt --quiet

Note: you may need to restart the kernel to use updated packages.
CPU times: user 28.8 ms, sys: 7.83 ms, total: 36.7 ms
Wall time: 1.74 s


In [4]:
# === Built-in ===
import os
import sys
import datetime
from pathlib import Path
from typing import List
import mlflow
import logging

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../")))

# === Third-party libraries ===
import pandas as pd
import warnings
import yaml
from IPython import get_ipython
from IPython.display import display, Markdown, Code

# === Langchain ===
from langchain.prompts import ChatPromptTemplate
from langchain.schema import Document, StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings

# === Internal modules ===
 
# Utils
from src.utils import (
    load_config,
    load_secrets,
    load_secrets_to_env,
    configure_proxy,
    initialize_llm,
    configure_hf_cache,
    clean_code,
    generate_code_with_retries,
    get_context_window,
    dynamic_retriever,
    format_docs_with_adaptive_context
)

# Import utility functions and CodeGenerationService
from core.code_generation_service import CodeGenerationService

/opt/conda/lib/python3.12/site-packages/mlflow/pyfunc/utils/data_validation.py:168: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(
/opt/conda/lib/python3.12/site-packages/mlflow/pyfunc/model.py:172: UserWarning: Type hint used in the model's predict function is not supported for MLflow's schema validation. Type hints must be wrapped in list[...] because MLflow assumes the predict method to take multiple input instances. Specify your type hint as `list[typing.Dict[str, typing.Any]]` for a valid signature. Remove the type hint to disable this warning. To enable validation for the input data, specify input example or model signature when logging the model. 
  func_info = _get_func_info_if_type_hint_supported(predict_attr)


# Configure Settings

In [5]:
# ------------------------ Suppress Verbose Logs ------------------------
warnings.filterwarnings("ignore")

In [6]:
# === Configuration settings ===
CONFIG_PATH = "../configs/config.yaml"
SECRETS_PATH = "../configs/secrets.yaml"
DEMO_FOLDER = "../demo"
MLFLOW_MODEL_NAME = "Code-Generation-Model"
LOCAL_MODEL_PATH = "/home/jovyan/datafabric/meta-llama3.1-8b-Q8/Meta-Llama-3.1-8B-Instruct-Q8_0.gguf"

# Define embedding model parameters
EMBEDDING_MODEL_NAME = "all-MiniLM-L6-v2"
EMBEDDING_MODEL_PATH = os.path.join(
    os.environ.get("HF_HOME", "/home/jovyan/local/hugging_face"),
    "embedding_models", 
    EMBEDDING_MODEL_NAME
)

In [7]:
# Configure HuggingFace cache for the embedding model
configure_hf_cache()

## Configuration and Secrets Loading

In this section, we load configuration parameters and API keys from separate YAML files. This separation helps maintain security by keeping sensitive information (API keys) separate from configuration settings.

- **config.yaml**: Contains non-sensitive configuration parameters like model sources and URLs
- **secrets.yaml**: Contains sensitive API keys for services like HuggingFace
- *(Optional for Premium users)* Secrets such as API keys for services like HuggingFace can be stored as environment variables for the project and loaded into the notebook (see the project's README file for steps on how to save secrets in Secrets Manager).

In [8]:
# Load secrets from secrets.yaml file (if it exists) into environment
if Path(SECRETS_PATH).exists():
    load_secrets_to_env(SECRETS_PATH)
else:
    print(f"No secrets file found at {SECRETS_PATH}; relying on preexisting environment")

# Retrieve secrets from environment
try:
    secrets = load_secrets()
except ValueError:
    secrets = {}

# Load configuration and secrets
config = load_config(CONFIG_PATH)

print("✅ Configuration loaded successfully")
print("✅ Secrets loaded successfully")

No secrets file found at ../configs/secrets.yaml; relying on preexisting environment
✅ Configuration loaded successfully
✅ Secrets loaded successfully


# Register and Log the Model to MLFlow

In [9]:
%%time

mlflow.set_tracking_uri('/phoenix/mlflow')
# Set up the MLflow experiment
mlflow.set_experiment(MLFLOW_MODEL_NAME)

# Check if the model file exists
if not os.path.exists(LOCAL_MODEL_PATH):
    logger.info(f"⚠️ Warning: Model file not found at {LOCAL_MODEL_PATH}. Please verify the path.")

# Verify the locally saved embedding model exists
if not os.path.exists(EMBEDDING_MODEL_PATH):
    # If the embedding model wasn't saved earlier, do it now
    try:
        from sentence_transformers import SentenceTransformer
        os.makedirs(os.path.dirname(EMBEDDING_MODEL_PATH), exist_ok=True)
        logger.info(f"Downloading and saving embedding model to {EMBEDDING_MODEL_PATH}...")
        model = SentenceTransformer(EMBEDDING_MODEL_NAME)
        model.save(EMBEDDING_MODEL_PATH)
        logger.info(f"Embedding model saved successfully.")
    except Exception as e:
        logger.error(f"Error saving embedding model: {str(e)}")
        logger.warning("Will proceed without a local embedding model. The service will download it during initialization.")
        EMBEDDING_MODEL_PATH = None

# Use the CodeGenerationService's log_model method to register the model in MLflow
with mlflow.start_run(run_name=MLFLOW_MODEL_NAME) as run:
    # Log and register the model using the service's classmethod
    CodeGenerationService.log_model(
        artifact_path="code_generation_service",
        config_path=CONFIG_PATH,
        secrets_dict=secrets if secrets else None,
        model_path=LOCAL_MODEL_PATH,
        embedding_model_path=EMBEDDING_MODEL_PATH,
        delay_async_init=True,  # Since this service requires parallelism we use this parameter to avoid error when picling the model
        demo_folder=DEMO_FOLDER
    )
    
    # Register the model in MLflow Model Registry
    model_uri = f"runs:/{run.info.run_id}/code_generation_service"
    mlflow.register_model(
        model_uri=model_uri,
        name=MLFLOW_MODEL_NAME
    )
    
    logger.info(f"✅ Model registered successfully with run ID: {run.info.run_id}")

2025-08-08 11:56:03,296 - INFO - Using local embedding model from: /home/jovyan/local/hugging_face/embedding_models/all-MiniLM-L6-v2
2025/08/08 11:56:03 INFO mlflow.models.signature: Inferring model signature from type hints
/opt/conda/lib/python3.12/site-packages/mlflow/pyfunc/__init__.py:3139: UserWarning: Failed to infer signature from type hint: Type hints must be wrapped in list[...] because MLflow assumes the predict method to take multiple input instances. Specify your type hint as `list[typing.Dict[str, typing.Any]]` for a valid signature.
  signature_from_type_hints = _infer_signature_from_type_hints(


2025-08-08 11:58:14,490 - INFO - Model and artifacts successfully registered in MLflow.
Registered model 'Code-Generation-Model' already exists. Creating a new version of this model...
Created version '16' of model 'Code-Generation-Model'.
2025-08-08 11:58:15 - INFO - ✅ Model registered successfully with run ID: 12dd903e9ea2469f8f292d2f3c802940


CPU times: user 819 ms, sys: 16 s, total: 16.8 s
Wall time: 2min 12s


### API Usage Examples

The optimized code generation service can be invoked through its REST API using the following patterns:

#### Details

The API includes some features like:

1. **metadata_only mode**: Allows extracting only basic metadata without running the resource-intensive LLM analysis
2. **Timeout controls**: Prevents worker timeouts with configurable processing limits

```bash
# Example 1: Direct code generation (no repository)
curl -X 'POST' \
  'https://endpoint/invocations' \
  -H 'accept: application/json' \
  -H 'Content-Type: application/json' \
  -d '{
  "inputs": {
    "question": "Write Python code to load the LLM model using Ollama with \'llama3\' and generate an inspirational quote."
  }
}'

# Example 2: Repository-enhanced code generation with metadata-only mode (FAST)
curl -X 'POST' \
  'https://endpoint/invocations' \
  -H 'accept: application/json' \
  -H 'Content-Type: application/json' \
  -d '{
  "inputs": {
    "question": "Write Python code to scrape a website and extract all links",
    "repository_url": "https://github.com/MunGell/awesome-for-beginners"
  }
}'

# Example 3: Repository-enhanced code generation with full processing (DEEP UNDERSTANDING)
curl -X 'POST' \
  'https://endpoint/invocations' \
  -H 'accept: application/json' \
  -H 'Content-Type: application/json' \
  -d '{
  "inputs": {
    "question": "Write Python code to scrape a website and extract all links",
    "repository_url": "https://github.com/MunGell/awesome-for-beginners"
  }
}'
```

The service will automatically adapt to the parameters provided:
- With `metadata_only=true`, it performs lightweight processing ideal for large repositories
- With full processing, it performs deep analysis but take longer

In [10]:
end_time: float = time.time()
elapsed_time: float = end_time - start_time
elapsed_minutes: int = int(elapsed_time // 60)
elapsed_seconds: float = elapsed_time % 60

logger.info(f"⏱️ Total execution time: {elapsed_minutes}m {elapsed_seconds:.2f}s")
logger.info("✅ Notebook execution completed successfully.")

2025-08-08 11:58:15 - INFO - ⏱️ Total execution time: 2m 17.09s
2025-08-08 11:58:15 - INFO - ✅ Notebook execution completed successfully.


Built with ❤️ using [**HP AI Studio**](https://hp.com/ai-studio).